In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import os   
import sys
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

from scipy.optimize import minimize
from scipy import interpolate

from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
import warnings
from scipy.optimize import fsolve
from scipy.stats import norm

from sklearn.decomposition import PCA
import warnings
from scipy.optimize import fsolve
import numpy.polynomial.polynomial as poly
warnings.filterwarnings(action='ignore')

In [4]:
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
years = ['2019', '2020', '2021', '2022', '2023']


In [15]:
import requests
api_key = "UVOO8K6JL3BPN3OO"
symbols = ['AAPL', 'MSFT', 'AMZN', 'TSLA', 'META', 'NFLX', 'NVDA', 'AMD', 'INTC']
all_df = []
for symbol in symbols:
    print(symbol)
    daily_variance = []
    for year in years:
        for month in months:
            url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=5min&apikey={api_key}&outputsize=full&month={year}-{month}&datatype=json"
            response = requests.get(url)
            data = response.json()
            # Extract the time series data
            time_series = data['Time Series (5min)']
            # Convert the time series data to a DataFrame
            df = pd.DataFrame.from_dict(time_series, orient='index')
            # Convert the index to datetime
            df.index = pd.to_datetime(df.index)
            df['date'] = df.index.date
            daily = df[['4. close', 'date']].groupby(['date']).apply(lambda x:x.astype('float64').pct_change().var())
            daily_variance.append(daily)

    var_df = pd.concat(daily_variance).rename({'4. close': 'Variance'})
    var_df['ticker'] = symbol
    all_df.append(var_df)

AAPL
MSFT
AMZN
TSLA
META
NFLX
NVDA
AMD
INTC


In [16]:
final_df = pd.concat(all_df)

In [21]:
final_df

,4. close,ticker
date,,
2019-01-02,0.000044,AAPL
2019-01-03,0.000005,AAPL
2019-01-04,0.000002,AAPL
2019-01-07,0.000003,AAPL
2019-01-08,0.000002,AAPL
...,...,...
2023-12-22,0.000002,INTC
2023-12-26,0.000004,INTC
2023-12-27,0.000036,INTC


In [24]:
final_df.reset_index().pivot(index='date', columns='ticker', values='4. close').to_csv('daily_variance_temp.csv')   